In [ ]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision.transforms import ToTensor 
import os

In [ ]:
path1 = "/content/drive/MyDrive/Dataset"
path2 = "/content/drive/MyDrive/Models/Pytorch/"

In [ ]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
print(f"available {device} device ")

device = torch.device(device)

class FeedForwardNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential(
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10 )
    )
    self.softmax = nn.Softmax(dim=1)
  
  def forward(self, input_data):  #indicates pytorch how to manipulate the data
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return predictions

def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader:
    inputs , targets = inputs.to(device), targets.to(device)

    # calculate loss
    preds = model(inputs)
    loss = loss_fn(preds,targets)

    # backpropagate loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  print(f"loss : {loss.item()}")
 
def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range(epochs):
    print(f"epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("------------------------")
  print("training done")

def download_mnist_datasets(path,download=True):
  train_data = datasets.MNIST(
      root=path,
      download=download,
      train = True,
      transform = ToTensor()
  ) 
  validation_data = datasets.MNIST(
      root=path,
      download=download,
      train = False,
      transform = ToTensor()
  ) 
  return train_data, validation_data


available cuda:0 device 


In [ ]:
train_data , _ = download_mnist_datasets(path="/content/drive/MyDrive/Dataset")
print("MNIST dataset downloaded")

MNIST dataset downloaded


In [ ]:
BATCH = 128
EPOCHS = 100
LEARNING_RATE = 1e-5

In [ ]:
feed_forward_net = FeedForwardNet()
feed_forward_net.cuda()

FeedForwardNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense_layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimiser = torch.optim.Adam(feed_forward_net.parameters(), lr =LEARNING_RATE )

In [ ]:
# create a dataloader 
train_data_loader = DataLoader(train_data,batch_size= BATCH)


In [ ]:
train(feed_forward_net, train_data_loader, loss_fn,optimiser, device, EPOCHS)
torch.save(feed_forward_net.state_dict(), "/content/drive/MyDrive/Models/Pytorch/feedforward.pth")
print("Model Is trained")

loss : 1.5054715871810913
------------------------
epoch 89
loss : 1.5050454139709473
------------------------
epoch 90
loss : 1.5046306848526
------------------------
epoch 91
loss : 1.5042308568954468
------------------------
epoch 92
loss : 1.503841757774353
------------------------
epoch 93
loss : 1.5034593343734741
------------------------
epoch 94
loss : 1.5030831098556519
------------------------
epoch 95
loss : 1.5027211904525757
------------------------
epoch 96
loss : 1.5023651123046875
------------------------
epoch 97
loss : 1.502016544342041
------------------------
epoch 98
loss : 1.5016741752624512
------------------------
epoch 99
loss : 1.5013419389724731
------------------------
epoch 100
loss : 1.5010143518447876
------------------------
training done
Model Is trained


In [ ]:
class_mapping = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]
def predict(model,input, target, class_mapping):
  model.eval()            # with this .eval() method, 
  with torch.no_grad():
    predicted = model(input)   # preds are of shape tensor(1,10) -> [[0.1, 0.1, 0.2, 0.89, 0.2,.....]]
    predicted_index = predicted[0].argmax(0)
    predicted = class_mapping[predicted_index]
    expected = class_mapping[target]
  return predicted, expected

In [ ]:

# inferencing steps
#Load back the model
feed_forward_instance = FeedForwardNet()
state_dict = torch.load(os.path.join(path2,'feedforward.pth'))
feed_forward_instance.load_state_dict(state_dict)
#get validation data
_, validation_data = download_mnist_datasets(path = path1)

 


In [ ]:
validaton_data_loader = DataLoader(validation_data,100)
for input , target in validaton_data_loader:
  for input , target in zip(input, target):#nput, target = validation_data[0][0], validation_data[0][1]
#make inference
    preds, expect = predict(feed_forward_instance, input, target, class_mapping)
    print(f"expected {expect} ,predicted {preds}")
    
  break

expected 7 ,predicted 7
expected 2 ,predicted 2
expected 1 ,predicted 1
expected 0 ,predicted 0
expected 4 ,predicted 4
expected 1 ,predicted 1
expected 4 ,predicted 4
expected 9 ,predicted 9
expected 5 ,predicted 6
expected 9 ,predicted 9
expected 0 ,predicted 0
expected 6 ,predicted 6
expected 9 ,predicted 9
expected 0 ,predicted 0
expected 1 ,predicted 1
expected 5 ,predicted 5
expected 9 ,predicted 9
expected 7 ,predicted 7
expected 3 ,predicted 3
expected 4 ,predicted 4
expected 9 ,predicted 9
expected 6 ,predicted 6
expected 6 ,predicted 6
expected 5 ,predicted 5
expected 4 ,predicted 4
expected 0 ,predicted 0
expected 7 ,predicted 7
expected 4 ,predicted 4
expected 0 ,predicted 0
expected 1 ,predicted 1
expected 3 ,predicted 3
expected 1 ,predicted 1
expected 3 ,predicted 3
expected 4 ,predicted 0
expected 7 ,predicted 7
expected 2 ,predicted 2
expected 7 ,predicted 7
expected 1 ,predicted 1
expected 2 ,predicted 2
expected 1 ,predicted 1
expected 1 ,predicted 1
expected 7 ,pred